In [1]:
import pandas as pd
import numpy as np
from math import *
import matplotlib.pyplot as plt
import matplotlib
import matplotlib
import scipy.stats

In [34]:
colorectal = pd.read_csv('data/Colorectal TCGA.csv')
lung_adeno = pd.read_csv('data/Lung Adenocarcinoma (TCGA, Provisional).csv')
lung_squa = pd.read_csv('data/Lung Squamous Cell Carcinoma (TCGA, Provisional).csv')

In [35]:
ces_colo = pd.read_table('data/expr_ces.txt')
ces_adeno = pd.read_table('data/lung_ces_expr.txt')
ces_squa = pd.read_table('data/lung2_ces_expr.txt')

In [36]:
msi_colo = pd.read_table('data/colorectal_msi_expr.txt').transpose()[2:]
msi_adeno = pd.read_table('data/lungAdeno_msi_expr.txt').transpose()[2:]
msi_squa = pd.read_table('data/lungSqua_msi_expr.txt').transpose()[2:]
msi_colo.columns = pd.read_table('data/colorectal_msi_expr.txt')['COMMON']
msi_adeno.columns = pd.read_table('data/lungAdeno_msi_expr.txt')['COMMON']
msi_squa.columns = pd.read_table('data/lungSqua_msi_expr.txt')['COMMON']

In [37]:
msi_colo.head()

COMMON,AHR,AMFR,ANG,ATP5A1,CALM1,CASP2,CD55,CFTR,CRIP1,CSNK1E,...,STK38,TCF7,TDGF1,TFF2,TGFBR2,TNNT1,TSC22D1,TYMS,USP14,YWHAB
TCGA-D5-6531-01,-0.2984,-0.6495,-0.8204,0.9265,-1.7234,2.875,-0.4922,2.1094,-0.8707,-0.2133,...,-0.4487,7.7511,-0.4772,-0.1796,-0.809,-0.3135,1.8998,-0.081,1.0918,2.652
TCGA-DM-A28A-01,-1.0773,-0.5238,-0.5854,-2.0746,-0.7818,0.0617,-0.6551,-0.7188,-0.8907,-1.3379,...,-0.4353,-0.4304,1.8356,-0.1828,-0.6583,-0.388,1.3653,-0.741,-1.7288,9.92
TCGA-DC-5869-01,0.375,-0.186,-0.6932,-1.7211,-1.31,-0.8216,-0.468,0.1565,-0.826,-1.3158,...,-0.4917,0.0005,0.7481,-0.1817,0.2413,-0.4003,0.8628,-1.1431,-0.9504,7.6206
TCGA-F4-6460-01,-0.326,0.6383,-0.3655,-2.3986,-0.5795,1.736,-0.7927,-0.6323,-0.7709,-0.1976,...,-0.0009,2.3799,0.2488,-0.1712,0.0434,-0.4039,1.0374,-1.2082,-2.7208,2.924
TCGA-DY-A1DC-01,-0.4386,0.765,-0.4288,-1.8909,0.1764,0.2216,-0.4758,0.4233,-0.6868,-0.4071,...,0.0015,0.3152,0.2526,-0.1793,0.2781,-0.2171,-0.0876,-1.1852,-2.4082,4.7469


# MSI / MSS categorization

In [38]:
upreg = ['ANG','TFF2','SRSF6','SET','EIF5A','RARRES1' ,'CRIP1','CD55','IQGAP2','P4HA1',
         'TYMS','GLRX','NUCB2','FUT8','HNRNPL','PCCB','EIF5A','POLR2L','GTF2A2','GALNT1',
         'NDUFA9','TNNT1','AHR','PRKAR2B','ATP5A1','AMFR','OSTF1','CALM1','MANF','EWSR1',
         'SSR1','MFAP1','GNAQ','PPP2R5E','USP14','GCH1','KIF11','CASP2','SLC1A1','PGGT1B','RAB27B']
downreg = ['FTO','PLAGL1','STK38','DMD','NONO','CSNK1E','TCF7','POFUT1','OCRL','ELF1','LRP1','MLH1',
           'MARCKS','JUN','ELN','RAB32','TSC22D1','YWHAB','SEMA3C','GABRE','NEK3','PRKCI','TGFBR2',
           'FCGRT','CFTR','SERINC3','HSPH1','TDGF1','CYP2B7P']

In [77]:
def mrr_cat(df_msi):
    df = pd.DataFrame()
    for ix,row in df_msi.iterrows():
        up = row[upreg].sum()
        down = row[downreg].sum()
    
        if up > 0 and down < 0:
            cat = 'msi'
            label = 0
        elif up < 0 and down > 0:
            cat = 'mss'
            label = 1
        else:
            cat = 'unknown'
            label = 2 
        df = df.append(pd.DataFrame([[ix,label,cat]]))
    df.columns = ['id','cat','label']
    return df

In [78]:
df = mrr_cat(msi_colo)
df.head()

,id,cat,label
0,TCGA-D5-6531-01,1,mss
0,TCGA-DM-A28A-01,1,mss
0,TCGA-DC-5869-01,1,mss
0,TCGA-F4-6460-01,1,mss
0,TCGA-DY-A1DC-01,1,mss


In [ ]:
for ix,p in results.iterrows():
    if p['upscore']>0 and p['downscore']<0:
        cat = 'msi'
        label = 0
    elif p['upscore']<0 and p['downscore']>0:
        cat = 'mss'
        label = 1
    else:
        cat = 'unknown'
        label = 2
        
    results = results.set_value(ix,'cat',cat)
    results = results.set_value(ix,'label',label)

results.head()

In [ ]:
fig, ax = plt.subplots()
colors = ['red','green','blue']
ax.scatter(results['upscore'], results['downscore'],c = results['label'],cmap = matplotlib.colors.ListedColormap(colors))
plt.xlabel('Up score')
plt.ylabel('Down score')
ax.grid(True, which='both')

ax.axhline(y=0, color='k')
ax.axvline(x=0, color='k')
plt.title('Visualisation of MMR classification of patients')
plt.show()

In [ ]:
def show_plots(df_full, df_ces, name):
    ces = []
    for p in df_ces.columns[2:]:
        ces.append([p,df_ces[p].sum()])
    ces = pd.DataFrame(ces)
    ces.columns = ['id','ces']  
    df = df_full.merge(ces,how='inner',left_on='SAMPLE ID', right_on='id')

    # First plot : CNA vs. Mutation Count
    fig, ax = plt.subplots()
    ax.scatter(df['CNA'], df['Mutation Count'])
    plt.xlabel('CNA')
    plt.ylabel('Mutation Count')
    plt.title('CNA and Mutation in '+name+' cancer')
    plt.show()
    
    # Second plot : CES vs. Mutation
    plt.subplot(1,2,1)
    plt.scatter(df['ces'], df['Mutation Count'])
    plt.xlabel('CES')
    plt.ylabel('Mutation Count')
    #plt.ylim(0,1400)
    plt.title('CES and Mutation')

    plt.subplot(1,2,2)
    plt.scatter(df_lung['ces'], df_lung['CNA'])
    plt.xlabel('CES')
    plt.ylabel('CNA')
    # plt.ylim(0,1600)
    plt.title('CES and CNA')
    plt.tight_layout()
    plt.show()